In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error

import warnings
warnings.simplefilter("ignore", category=DeprecationWarning)

In [2]:
def obtener_rmse(col_true, col_pred):
    return mean_squared_error(col_true, col_pred)**0.5

In [3]:
propiedades = pd.read_csv('set_datos_propiedades.csv')

In [4]:
propiedades = propiedades.loc[(propiedades.price_aprox_usd.notnull()) & (propiedades.superficie.notnull()),\
                             ['place_name_encoded', 'property_type_encoded','price_aprox_usd','superficie',\
                             'Year','Month','seguridad','aire','gimnasio','cochera','pileta']]

In [5]:
propiedades.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1132495 entries, 0 to 1413024
Data columns (total 11 columns):
place_name_encoded       1132495 non-null int64
property_type_encoded    1132495 non-null int64
price_aprox_usd          1132495 non-null float64
superficie               1132495 non-null float64
Year                     1132495 non-null int64
Month                    1132495 non-null int64
seguridad                1132495 non-null bool
aire                     1132495 non-null bool
gimnasio                 1132495 non-null bool
cochera                  1132495 non-null bool
pileta                   1132495 non-null bool
dtypes: bool(5), float64(2), int64(4)
memory usage: 65.9 MB


# Gradient boosting regressor

In [6]:
columnas = ['superficie','place_name_encoded','property_type_encoded','seguridad','gimnasio', 'aire', 'pileta', 'cochera']
columnas_precio = columnas + ['price_aprox_usd']

In [7]:
set_entrenamiento = propiedades.loc[(propiedades.Year >= 2016) &((propiedades.Year < 2017) | (propiedades.Month < 6))\
                                    ,columnas_precio]
set_pruebas = propiedades.loc[(propiedades.Year == 2017) & (propiedades.Month == 6),columnas_precio].head(20000)

set_entrenamiento_datos = set_entrenamiento.loc[:,columnas]
set_entrenamiento_resultado = set_entrenamiento.loc[:,'price_aprox_usd']

In [8]:
gradient = GradientBoostingRegressor()
gradient.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply(lambda x: gradient.predict([x])[0],axis = 1)
precision = gradient.score(set_pruebas.loc[:,columnas], set_pruebas.price_aprox_usd) * 100
error = obtener_rmse(set_pruebas.price_aprox_usd, set_pruebas.resultado)
print("Precision = {:.2f} % , error = {}".format(precision, error))

Precision = 49.59 % , error = 263717.911261235


#### Al correrlo varias veces con distintos valores, fue mejorando la precision. Despues agrego un for con varios valores para ver como mejora

## Ahora que tenemos una intuicion, probamos cambiando los parametros

In [9]:
columnas = ['superficie','place_name_encoded','property_type_encoded','seguridad','gimnasio', 'aire', 'pileta', 'cochera']
columnas_precio = columnas + ['price_aprox_usd']

In [10]:
set_entrenamiento = propiedades.loc[(propiedades.Year >= 2016) &((propiedades.Year < 2017) | (propiedades.Month < 6))\
                                    ,columnas_precio]
set_pruebas = propiedades.loc[(propiedades.Year == 2017) & (propiedades.Month == 6),columnas_precio].head(20000)

set_entrenamiento_datos = set_entrenamiento.loc[:,columnas]
set_entrenamiento_resultado = set_entrenamiento.loc[:,'price_aprox_usd']


In [15]:
res = []
loss = ['ls', 'lad', 'huber', 'quantile']
n_estimators = [300, 500, 1000, 2000]

for l in loss:
    for n in n_estimators:
        gradient = GradientBoostingRegressor(loss = l, n_estimators = n)
        gradient.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
        set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply(lambda x: gradient.predict([x])[0],axis = 1)
        precision = gradient.score(set_pruebas.loc[:,columnas],set_pruebas.loc[:,'price_aprox_usd']) * 100
        error = obtener_rmse(set_pruebas.price_aprox_usd, set_pruebas.resultado)
        res.append((l, n, precision, error))
        print(l,' - ', n)

ls  -  300
ls  -  500
ls  -  1000
ls  -  2000
lad  -  300
lad  -  500
lad  -  1000
lad  -  2000
huber  -  300
huber  -  500
huber  -  1000
huber  -  2000
quantile  -  300
quantile  -  500
quantile  -  1000
quantile  -  2000


In [18]:
for r in res

    print("Loss = {}, n_estimators = {}, precision = {:.2f} % , error = {}".format(r[0],r[1],r[2],r[3]))

Loss = ls, n_estimators = 300, precision = 55.15 % , error = 248737.31486991755
Loss = ls, n_estimators = 500, precision = 57.50 % , error = 242135.99581654233
Loss = ls, n_estimators = 1000, precision = 60.48 % , error = 233486.84277362804
Loss = ls, n_estimators = 2000, precision = 62.91 % , error = 226215.10776356
Loss = lad, n_estimators = 300, precision = 47.35 % , error = 269518.0659149018
Loss = lad, n_estimators = 500, precision = 49.66 % , error = 263524.37826822215
Loss = lad, n_estimators = 1000, precision = 50.67 % , error = 260861.96525934708
Loss = lad, n_estimators = 2000, precision = 51.67 % , error = 258216.88529997237
Loss = huber, n_estimators = 300, precision = 51.41 % , error = 258917.110961643
Loss = huber, n_estimators = 500, precision = 46.40 % , error = 271928.52928042517
Loss = huber, n_estimators = 1000, precision = 47.81 % , error = 268320.914070446
Loss = huber, n_estimators = 2000, precision = 48.50 % , error = 266545.7580620435
Loss = quantile, n_estimato

In [32]:
min_error = float('inf')
max_precision = 0
tupla_min_error = ()
tupla_max_precision = ()
for r in res:
    if r[3] < min_error:
        min_error = r[3]
        tupla_min_error = r
    if abs(r[2]) > max_precision:
        max_precision = r[2]
        tupla_max_precision = r
        
print("Mayor precision = Loss = {}, n_estimators = {}, precision = {:.2f} % , error = {}".\
              format(tupla_max_precision[0],tupla_max_precision[1],tupla_max_precision[2],tupla_max_precision[3]))
print("Menor error = Loss = {}, n_estimators = {}, precision = {:.2f} % , error = {}".\
              format(tupla_min_error[0],tupla_min_error[1],tupla_min_error[2],tupla_min_error[3]))

Mayor precision = Loss = ls, n_estimators = 2000, precision = 62.91 % , error = 226215.10776356
Menor error = Loss = ls, n_estimators = 2000, precision = 62.91 % , error = 226215.10776356


## Usamos ls como loss

In [33]:
columnas = ['superficie','place_name_encoded','property_type_encoded','seguridad','gimnasio', 'aire', 'pileta', 'cochera']
columnas_precio = columnas + ['price_aprox_usd']

In [34]:
set_entrenamiento = propiedades.loc[(propiedades.Year >= 2016) &((propiedades.Year < 2017) | (propiedades.Month < 6))\
                                    ,columnas_precio]
set_pruebas = propiedades.loc[(propiedades.Year == 2017) & (propiedades.Month == 6),columnas_precio].head(20000)

set_entrenamiento_datos = set_entrenamiento.loc[:,columnas]
set_entrenamiento_resultado = set_entrenamiento.loc[:,'price_aprox_usd']

In [36]:
loss = 'ls'
n_est = 2000

In [37]:
gradient = GradientBoostingRegressor(loss = loss, n_estimators = n_est, warm_start = True)
for i in range(10):
    gradient.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
    set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply(lambda x: gradient.predict([x])[0],axis = 1)
    precision = gradient.score(set_pruebas.loc[:,columnas],set_pruebas.price_aprox_usd) * 100
    error = obtener_rmse(set_pruebas.price_aprox_usd,set_pruebas.resultado)
    print("Precision = {:.2f} % , error = {}".format(precision, error))

Precision = 62.91 % , error = 226215.10776355996
Precision = 62.91 % , error = 226215.10776355996
Precision = 62.91 % , error = 226215.10776355996
Precision = 62.91 % , error = 226215.10776355996
Precision = 62.91 % , error = 226215.10776355996
Precision = 62.91 % , error = 226215.10776355996
Precision = 62.91 % , error = 226215.10776355996
Precision = 62.91 % , error = 226215.10776355996
Precision = 62.91 % , error = 226215.10776355996
Precision = 62.91 % , error = 226215.10776355996


#### No se obtuvo mejora utilizando warm_start = True, por lo que realizar varias iteraciones con los mismos datos no mejorará la precisión

In [38]:
gradient = GradientBoostingRegressor(loss = loss, n_estimators = 10000, warm_start = True)
gradient.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply(lambda x: gradient.predict([x])[0],axis = 1)
precision = gradient.score(set_pruebas.loc[:,columnas],set_pruebas.price_aprox_usd) * 100
error = obtener_rmse(set_pruebas.price_aprox_usd,set_pruebas.resultado)
print("Precision = {:.2f} % , error = {}".format(precision, error))

Precision = 68.10 % , error = 209782.67862516703


In [66]:
set_pruebas.resultado.describe()

count    2.000000e+04
mean     2.645389e+05
std      2.998174e+05
min      5.833006e+02
25%      1.081312e+05
50%      1.716801e+05
75%      2.996141e+05
max      6.980550e+06
Name: resultado, dtype: float64

In [48]:
set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,"resultado"].apply(lambda x: abs(x))

In [51]:
obtener_rmse(set_pruebas.price_aprox_usd,set_pruebas.resultado)

209673.22218335391

## Tomo un millon de estimadores para ver cuanto mejora la precision

In [ ]:
columnas = ['superficie','place_name_encoded','property_type_encoded','seguridad','gimnasio', 'aire', 'pileta', 'cochera']
columnas_precio = columnas + ['price_aprox_usd']

In [ ]:
set_entrenamiento = propiedades.loc[(propiedades.Year >= 2016) &((propiedades.Year < 2017) | (propiedades.Month < 6))\
                                    ,columnas_precio]
set_pruebas = propiedades.loc[(propiedades.Year == 2017) & (propiedades.Month == 6),columnas_precio].head(20000)

set_entrenamiento_datos = set_entrenamiento.loc[:,columnas]
set_entrenamiento_resultado = set_entrenamiento.loc[:,'price_aprox_usd']

In [ ]:
gradient = GradientBoostingRegressor(loss = loss, n_estimators = 1000000, warm_start = True)
gradient.fit(set_entrenamiento_datos,set_entrenamiento_resultado)
set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,columnas].apply(lambda x: gradient.predict([x])[0],axis = 1)
precision = gradient.score(set_pruebas.loc[:,columnas],set_pruebas.price_aprox_usd) * 100
error = obtener_rmse(set_pruebas.price_aprox_usd,set_pruebas.resultado)
print("Precision = {:.2f} % , error = {}".format(precision, error))

In [ ]:
set_pruebas.resultado.describe()

In [ ]:
set_pruebas.loc[:,'resultado'] = set_pruebas.loc[:,"resultado"].apply(lambda x: abs(x))

In [ ]:
obtener_rmse(set_pruebas.price_aprox_usd,set_pruebas.resultado)

# calculo con los verdaderos datos a analizar

In [43]:
analizar = pd.read_csv("properati_dataset_modificado.csv")

In [44]:
analizar.loc[:,'price_usd'] = analizar.loc[:,columnas].apply(lambda x: gradient.predict([x])[0],axis = 1)

In [54]:
analizar.price_usd.describe()

count    1.416600e+04
mean     2.391993e+05
std      3.214759e+05
min     -2.004290e+06
25%      1.115106e+05
50%      1.636383e+05
75%      2.719184e+05
max      1.960145e+07
Name: price_usd, dtype: float64

In [56]:
# Las predicciones con precio negativo las convierto en valores positivos
analizar.loc[:,'price_usd'] = analizar.loc[:,"price_usd"].apply(lambda x: abs(x))

In [57]:
analizar.price_usd.describe()

count    1.416600e+04
mean     2.454957e+05
std      3.166934e+05
min      1.439696e+03
25%      1.122131e+05
50%      1.636433e+05
75%      2.744900e+05
max      1.960145e+07
Name: price_usd, dtype: float64

In [58]:
resultado = analizar.loc[:,['id','price_usd']]

In [59]:
resultado.to_csv('Gradient_boosting_1.csv', index = False)